In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import os
os.chdir("/Users/darshmac/Documents/cdac/For DBDA/PML/Practice_Datasets")

In [6]:
dp = pd.read_csv("diabetes_prediction_dataset.csv")
dum_dia = pd.get_dummies(dp,drop_first=True)

In [8]:
dum_dia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   age                          100000 non-null  float64
 1   hypertension                 100000 non-null  int64  
 2   heart_disease                100000 non-null  int64  
 3   bmi                          100000 non-null  float64
 4   HbA1c_level                  100000 non-null  float64
 5   blood_glucose_level          100000 non-null  int64  
 6   diabetes                     100000 non-null  int64  
 7   gender_Male                  100000 non-null  uint8  
 8   gender_Other                 100000 non-null  uint8  
 9   smoking_history_current      100000 non-null  uint8  
 10  smoking_history_ever         100000 non-null  uint8  
 11  smoking_history_former       100000 non-null  uint8  
 12  smoking_history_never        100000 non-null  uint8  
 13  

In [9]:
X = dum_dia.drop('diabetes',axis=1)

In [10]:
y = dum_dia['diabetes']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=23,shuffle=True)

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75000, 13)
(75000,)
(25000, 13)
(25000,)


In [15]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [57]:
knn = KNeighborsClassifier()

In [58]:
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [59]:
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)

In [60]:
print(accuracy_score(y_test,y_pred))
print(log_loss(y_test,y_pred_prob))

0.95448
0.6983668649877083


In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [62]:
kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=23)

In [63]:
print(knn.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}


In [64]:
params = {'n_neighbors':np.arange(1,20,1)}

In [65]:
gcv = GridSearchCV(knn,param_grid=params,
                  cv = kfold,
                  scoring='accuracy')

In [66]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
             scoring='accuracy')

In [67]:
print(gcv.best_params_)
print(gcv.best_score_)

{'n_neighbors': 7}
0.9553699999999999


In [42]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [43]:
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)

In [44]:
print(accuracy_score(y_test,y_pred))
print(log_loss(y_test,y_pred_prob))

0.95464
0.5339017432983924


In [45]:
#pipeline
from sklearn.pipeline import Pipeline

In [46]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [47]:
std_scaler = StandardScaler()

In [48]:
#pipe for std scaler
pipe_std = Pipeline([('SCL',std_scaler),('KNN',knn)])

In [49]:
pipe_std.fit(X_train,y_train)

Pipeline(steps=[('SCL', StandardScaler()),
                ('KNN', KNeighborsClassifier(n_neighbors=7))])

In [50]:
y_pred = pipe_std.predict(X_test)

In [51]:
print(accuracy_score(y_test,y_pred))

0.96148


In [52]:
#for minmaxscaler
minmaxscl = MinMaxScaler()

In [53]:
pipmin = Pipeline([('MXSCL',minmaxscl),('KNN',knn)])

In [54]:
pipmin.fit(X_train,y_train)

Pipeline(steps=[('MXSCL', MinMaxScaler()),
                ('KNN', KNeighborsClassifier(n_neighbors=7))])

In [55]:
y_pred = pipmin.predict(X_test)

In [56]:
print(accuracy_score(y_test,y_pred))

0.96292
